In [2]:
!pip install -U langchain langchain-core langchain-google-genai python-dotenv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.9
    Uninstalling langchain-core-1.2.9:
      Successfully uninstalled langchain-core-1.2.9
  Attempting uninstall: langgraph
    Found existing installation: langgraph 1.0.7
    Uninstalling langgraph-1.0.7:
      Successfully uninstalled langgraph-1.0.7
  Attempting uninstall: langchain
    Found existing installation: langchain 1.2.8
    Uninstalling langchain-1.2.8:
      Successfully uninstalled langchain-1.2.8


In [3]:
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")

# Using Low Temp for consistent comparison
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.0)

Enter your Google API Key: ··········


In [4]:
task = " write a rejection email to a candidate"
print("====lazy prompt ====")
print(llm.invoke(task).content)

====lazy prompt ====
Writing a rejection email is never easy, but it's crucial to be professional, respectful, and clear. Here are a few options, ranging from a standard rejection after an interview to an earlier-stage rejection, along with key considerations.

---

**Key Considerations Before Sending:**

*   **Timeliness:** Send it as soon as a decision is made. Don't leave candidates hanging.
*   **Clarity:** Be unambiguous that they have not been selected.
*   **Professionalism:** Maintain a respectful and appreciative tone.
*   **Consistency:** Use a consistent process for all candidates.
*   **Legal Safety:** Avoid giving specific, potentially discriminatory reasons for rejection. Focus on the selected candidate's fit or the company's needs.
*   **Personalization:** Always use the candidate's name.
*   **No Feedback (Unless Policy Allows & You're Prepared):** Generally, it's safer not to offer specific feedback unless your company has a clear policy for it and you're trained to de

In [5]:
structured_prompt = """

#constraints
1.Be extremely brief(under 50 words).
2.do not say " we found someone better " . say " the role changed ".
3.sign off with "all the best , keep flying ".

#output fomat
plain text , no subject line
"""
print("===structured promt ===")
print(llm.invoke(task).content)

===structured promt ===
Writing a rejection email is never easy, but it's crucial to be professional, respectful, and clear. Here are a few options, ranging from a standard rejection after an interview to an earlier-stage rejection, along with key considerations.

---

**Key Considerations Before Sending:**

*   **Timeliness:** Send it as soon as a decision is made. Don't leave candidates hanging.
*   **Clarity:** Be unambiguous that they have not been selected.
*   **Professionalism:** Maintain a respectful and appreciative tone.
*   **Consistency:** Use a consistent process for all candidates.
*   **Legal Safety:** Avoid giving specific, potentially discriminatory reasons for rejection. Focus on the selected candidate's fit or the company's needs.
*   **Personalization:** Always use the candidate's name.
*   **No Feedback (Unless Policy Allows & You're Prepared):** Generally, it's safer not to offer specific feedback unless your company has a clear policy for it and you're trained to

In [6]:
prompt_zero = "combine 'angry' and ' hungry' into a funny new word"
print(f"zero-shot: {llm.invoke(prompt_zero).content}")

zero-shot: The most common and widely accepted word for this is **Hangry**!

It perfectly combines the "hun-" from hungry and the "-gry" from angry, and it's a very relatable and funny term.


In [7]:
prompt_few="""
combine words into a funny new word .give a sarcastic definition.
input : breakfast + lunch
output : brunch (an excuse to drink alcohol before noon)
input : chill + relax
output : chillax (what annoying people say when you have panic attacks)
input : angry + hungry
output :
"""
print(f"few-shot:{llm.invoke(prompt_few).content}")

few-shot:output : hangry (the scientific term for why you're about to commit a minor felony over a missing snack)


In [9]:
from langchain_core.prompts import ChatPromptTemplate , FewShotChatMessagePromptTemplate
examples = [
    {"input": "the internet is down.","output":"we are observing connectively latency."}
    ,{"input": "this code implies a bug.","output":"the logic suggests unintended behaviour."}
    ,{"input": "i hate this feature .","output":"this feature doesnt align with my pre-trained model."}
]

example_fmt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}"),
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_fmt,
    examples=examples)

final_prompt = ChatPromptTemplate.from_messages([
    ("system","you are a corpo-speak translator . rewrite the input to sound professional."),
    few_shot_prompt,
    ("human","{text}")
])

chain=final_prompt | llm

print(chain.invoke({"text": "this app sucks."}).content)

The current application is exhibiting suboptimal performance.


In [12]:
# Install dependencies (safe to re-run)
# !pip install -U langchain langchain-core langchain-google-genai python-dotenv

# Load environment
from dotenv import load_dotenv
load_dotenv()

import os, getpass

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")

# Imports
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Initialize LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.2
)

# Your structured prompt converted into template
prompt = ChatPromptTemplate.from_template("""
Role: You are a Senior Python Developer with expertise in writing clean, maintainable, and production-quality code.

Objective:
Write a Python function that reverses a given string.

Constraints:
- The implementation must use recursion.
- Do NOT use slicing (e.g., [::-1]).
- Do NOT use built-in reverse utilities.

Style Requirements:
- Include a comprehensive docstring following PEP 257 conventions.
- The docstring must describe:
    * Purpose of the function
    * Parameters
    * Return value
    * Base case explanation
    * Time and space complexity
- Write clear, readable, and idiomatic Python code.
- Include type hints.

Deliver only the Python function definition.
""")

# LCEL chain
chain = prompt | llm | StrOutputParser()

# Invoke
print(chain.invoke({}))


```python
def reverse_string(s: str) -> str:
    """
    Reverses a given string using recursion.

    This function takes an input string `s` and returns a new string with its
    characters in reverse order. It strictly adheres to a recursive implementation
    and avoids the use of string slicing for reversal (e.g., [::-1]) or
    built-in reverse utilities (e.g., `reversed()`, `list.reverse()`).

    Args:
        s: The input string to be reversed.

    Returns:
        The reversed string.

    Base Case Explanation:
        The recursion terminates when the input string `s` has a length of 0 or 1.
        In these scenarios, the string is considered inherently reversed (an empty
        string remains empty, and a single-character string is its own reverse).
        Therefore, the string `s` is returned directly, preventing infinite recursion.

    Time Complexity:
        O(n^2), where n is the length of the input string.
        This quadratic complexity arises because string 